## Lab 1 Part I: METARs
#### 9/7/2022


The following tutorial is the first of three parts of the Python portion of Lab 1.  In this part, we will focus on how to work with METAR data in python using the MetPy and Pandas modules.  As with every future lab, I will include a link to the documentation of each module that we introduce for the first time.
<br />
### Module Documentation
1. MetPy Metar Parsing Function: https://unidata.github.io/MetPy/latest/api/generated/metpy.io.parse_metar_file.html
2. Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
3. The datetime function from the datetime module: https://docs.python.org/3/library/datetime.html


<br /><br />

If you have any questions about the code below, always feel free to reach out to me at mpvossen@uwm.edu. I am always willing to further explain the code. <br /> <br />


1. In most things we do in atmospheric science, we can save ourselves time by importing code (called <i>modules</i>) that someone else has written.  In the section below, I load the Python modules we are going to need to complete this portion of the lab.

In [ ]:
#from the data reading capabilities of metpy (metpy.io) import the metar reading capability (parse_metar_file)
from metpy.io import parse_metar_file

#import the data storage for the metar data.  This package lays the data out in a table like format
import pandas as pd

#from the dates and time code(datetime), import the date and time reading capabilities (datetime).
from datetime import datetime

#from python's data import module (io) import the ability to read a string as a file.  This allows us to avoid downloading files which speeds things up and keeps your files storage clean.
from io import StringIO

#import the module to download files from the internet
import requests

<br /><br />
2. In the next part of the lab, we need to convert surface observation data between units since the data are not always in the desired units.  Below is a sample function for how to convert wind from knots to mph. <br />

In [ ]:
#here the function is defined.  The def command says to define the function of the name convert_knots_to_mph with the input variable of the name value
def convert_knots_to_mph(value):
    
    #this line causes the function to return a value.  Here I'm returning the input variable divided by 0.868976, the conversion factor between kt and MPH. We alternatively could multiply by 1.15 to get the same result.
    #note the indentation of this line - Python uses indentation to organize and execute its code.
    return value / 0.868976
    

<br /><br />In the section below, create a function to convert a temperature value from °C to °F.  Name the function convert_c_to_f. <br />

<br /><br />

3. Let's acquire some METAR data.  First, we will need to download the METAR data from the Unidata THREDDS server.  This code relies on the modules imported earlier in this notebook. When you run this code, it may take up to 10 seconds to run.  Note: If you want to get your own surface data, you would go to https://thredds-test.unidata.ucar.edu/thredds/catalog/noaaport/text/metar/catalog.html.<br /> <br />



In [ ]:
#Here I set the create the variable that holds the time in UTC for which we want the METAR data.
#datetime(year, month, day, hour)
file_time = datetime(2022,9,7,18)

#Here I build the url to get the data.  Most of the url stays the same but the metar file name changes based on the date and hour that it is valid for.
#right before the quote I have the letter f, which means the values in the quotes after are what is called a formatted string.
#The formatted string allows us to use the file time that we defined before to generate the string.  Below I use this file time when I have
#{file_time:%Y%m%d_%H}.  This stands for {time variable:datetime format}.  For the datetime format there are special codes that 
#tell the string which value to insert.  To see what each code stands for, refer to the datetime module's documentation.
url = f"https://thredds-test.unidata.ucar.edu/thredds/fileServer/noaaport/text/metar/metar_{file_time:%Y%m%d_%H}00.txt"

#using the request module use the get function to retrieve the raw website data from the url we defined above
web_data = requests.get(url)

#here we take the web data from before and pull of the content (.content) from the web_data object.  
#Then we take the content and decode it to something that we can use rather than the website html (.decode())
web_content = web_data.content.decode()

#Here we take the decoded web content from above and make it a file object.  This avoids downloading the data to your file system which
#speeds things up, and it keeps your file space from becoming cluttered.  Instead, this puts the METAR file directly to the RAM of the machine
data_file = StringIO(web_content)


#we now tell metpy to parse out the file we have downloaded (data_file).  Also Metpy only can get the day of the month
#from the METAR, so we need to specify the month (file_time.month) and year (file_time.year) from the file time that we set before
#or else it will assume the current month and year.  
metar_data = parse_metar_file(data_file, month = file_time.month, year=file_time.year)

#below you can see that the data is parse out and now is in a form that is similar to a table.  This is called a data frame.
#also in Jupyter you can display one variable by typing out the variable name like I did below. (Note: this does not work outside Jupyter)
#if you need to display multiple variables in a cell, you will need to use the print statement instead
metar_data


<br /><br />
3. We now have parsed our data parsed.  The data is stored in something that is called a Pandas DataFrame, which you can visualize to be just like a table of data that you would see in a textbook or an Excel spreadsheet.  There are column names and row names for the table that we can use to access various parts of the data.  

With the way that MetPy prases the METAR data, the row names are the station names and the column names are the observation quantity names.  This structure is useful because sometimes when working with METAR data we need to get an observation for a single location.  In the code below, I use Pandas' syntax to get O'Hare Airport's observations in our sample file.  Multiple times may appear since O'Hare may make multiple observations during the requested hour. <br />


In [ ]:
#set the site variable to a string of O'Hare's 4 letter identifier
site = "KORD"

#from the metar data frame (the metar_data variable) slice out the row (.loc[]) that has the index that is for the site we want (site) and save it to the variable station.
station = metar_data.loc[site]

#display the sliced data for O'Hare. The data may look different, but it is still setup the same as the cell above.
station

<br /> <br /> 
4. We can also parse out specific variables by using the syntax below. <br />

In [ ]:
#from the data that only contains the metar for KORD (station) slice out the column named "windspeed" and save it to the variable station_wind.  
#For columns we can just do the brackets and we don't need a function like the .loc() function that we needed before for the row.
station_wind = station["wind_speed"]

#display the variable that we saved the wind speed data from KORD to.
station_wind

<br /><br />
5. The parsed observation is in the standard METAR wind speed units of kt.  We can convert these units by using the conversion function we created before:
<br />

In [ ]:
#using the convert_knots_to_mph function that I defined before to convert the wind speed for KORD (station_wind) from knots to mph and save the output from the function to the variable station_wind_mph.
station_wind_mph = convert_knots_to_mph(station_wind)
#display the station wind speed that resulted from the function above
station_wind_mph

<br /><br />
6. In the code section below, parse out the temperature (air_temperature), dewpoint (dew_point_temperature), pressure (air_pressure_at_sea_level), wind speed (wind_speed), wind direction (wind_direction), and cloud coverage (cloud_coverage) for Madison (KMSN).  Display the output so you can use it to answer question 6 in the lab.  Be sure to convert temperature and dewpoint to the appropriate units.
<br /><br />

<br /><br />

You have now completed Part I of the Python portion of Lab 1.  Be sure to submit the fully rendered Jupyter Notebook on GitHub when you are finished.